# Agents
LangChain has a SQL Agent which provides a more flexible way of interacting with SQL Databases than a chain. 

The main advantages of using the SQL Agent are:

1. It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table).
2. It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
3. It can query the database as many times as needed to answer the user question.
4. It will save tokens by only retrieving the schema from relevant tables.



In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="Q&A_over_SQL_agent"

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


# Initialize the agent 

We'll use the SQLDatabaseToolkit to create a bunch of tools:

* Create and execute queries
* Check query syntax
* Retrieve table descriptions
  
... and more

In [5]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000028E110511E0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000028E110511E0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000028E110511E0>),
 QuerySQLCheckerTool(description='Use this tool to 

# System Prompt
We will also want to create a system prompt for our agent. This will consist of instructions for how to behave.

In [6]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""


In [7]:
system_message = SystemMessage(content=SQL_PREFIX)

# Initializing agent
First, get required package LangGraph


We will use a prebuilt LangGraph agent to build our agent



In [8]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

C:\Users\neural_ninja\AppData\Local\Temp\ipykernel_12968\2704392570.py:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [9]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h5YqPg7yPwPYBMGYTiFFsUPT', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 548, 'total_tokens': 560}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7f283272-5d9d-43ee-b170-141f8c441082-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_h5YqPg7yPwPYBMGYTiFFsUPT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 548, 'output_tokens': 12, 'total_tokens': 560})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='call_h5YqPg7yPwPYBMGYTiFFsUPT')]}}
----
{'agent': {'messages': [AIMessage(content='', additional

In [10]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="How many tracks are there in the album with ID 5?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j6k0zzBotzfAM2TUhOfzEUhK', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 554, 'total_tokens': 566}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e2d5237-77b4-4909-a196-1fa6e0ec8af4-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_j6k0zzBotzfAM2TUhOfzEUhK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 12, 'total_tokens': 566})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='call_j6k0zzBotzfAM2TUhOfzEUhK')]}}
----
{'agent': {'messages': [AIMessage(content='', additional

# Printing only the final output

In [11]:
def print_final_generation(agent_executor,user_input):
    last_agent_message = None

    # Stream the output from the agent executor
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content=user_input)]}
    ):
        # Check if the message is from the agent and contains content
        if 'agent' in s and 'messages' in s['agent']:
            for message in s['agent']['messages']:
                if isinstance(message, AIMessage) and message.content:
                    last_agent_message = message.content

    # Print the last agent message
    if last_agent_message:
        print(last_agent_message)

In [12]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    print_final_generation(agent_executor,user_input)

Goodbye!


Note that the agent executes multiple queries until it has the information it needs:

List available tables;
Retrieves the schema for three tables;
Queries multiple of the tables via a join operation.
The agent is then able to use the result of the final query to generate an answer to the original question.

The agent can similarly handle qualitative questions:

In [13]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8XBlvPL0ZbT0GQktZlGU8Fxh', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 546, 'total_tokens': 558}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f905cf32a9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a16473b5-7b0c-49d9-b81f-d6bea1f074e6-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_8XBlvPL0ZbT0GQktZlGU8Fxh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 546, 'output_tokens': 12, 'total_tokens': 558})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='call_8XBlvPL0ZbT0GQktZlGU8Fxh')]}}
----
{'agent': {'messages': [AIMessage(content='', additional